# Intal·lacions i importacions

In [1]:
import os
import tarfile
from pathlib import Path
import gzip
import shutil
import tsplib95
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt

## Càrrega de fitxers

In [ ]:
# Rutes
tar_path = Path('Datasets/ALL_tsp.tar')
extract_path = Path('Datasets/ALL_tsp')

if extract_path.exists():
    shutil.rmtree(extract_path)

extract_path.mkdir(parents=True, exist_ok=True)

with tarfile.open(tar_path, 'r') as tar:
    tar.extractall(path=extract_path)

gz_files = list(extract_path.glob('*.gz'))

for gz_file in gz_files:
    output_file = extract_path / gz_file.stem
    with gzip.open(gz_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    gz_file.unlink()

print("ALL_tsp.tar descomprimit i fitxers .gz eliminats.")

ALL_tsp.tar descomprimit i fitxers .gz eliminats.


In [3]:
# Llista pels grafs de test
graphs = []

# Carregar fitxers .tsp
tsp_files = list(extract_path.glob('*.tsp'))

for tsp_file in sorted(tsp_files)[:5]: # -- Limit per no crash Kernel --
    graph = tsplib95.load(tsp_file).get_graph()
    graphs.append(graph)
    print(f"Carregat: {graph.name}")

Carregat: a280
Carregat: ali535
Carregat: att48
Carregat: att532
Carregat: bayg29


In [ ]:
# Neteja de fitxers temporals
shutil.rmtree(extract_path)

# Funcions

In [5]:
def validate_tours(tours):
    valid_tours = []

    for i, path in enumerate(tours):
        G = graphs[i]
        num_nodes = G.number_of_nodes()

        expected_nodes = set(range(1, num_nodes+1))
        path_set = set(path)

        if path_set == expected_nodes and len(path) == num_nodes:
            print(f"Graph {i} ({G.name}): ✅ Path vàlid")
            valid_tours.append((G.name, path))
        else:
            print(f"Graph {i} ({G.name}): ❌ Path invàlid")

            missing = expected_nodes - path_set
            extra   = path_set - expected_nodes

            if missing:
                print(f"  - Falta algun node: {missing}")
            if extra:
                print(f"  - Nodes sobrants: {extra}")
            if len(path) != num_nodes:
                print(f"  - Longitud incorrecta: {len(path)} (esperat {num_nodes})")

    return valid_tours

In [6]:
def save_tours(valid_tours, filename):
    output_dir = Path("Datasets")
    output_dir.mkdir(exist_ok=True)

    output_file = output_dir / filename

    with open(output_file, "w") as f:
        for name, path in valid_tours:
            f.write(f"{name} : {path}\n")
    
    print(f"Tours saved at {output_file}")

# NN

In [ ]:
def nearest_neighbor_tour(G):
    nodes = list(G.nodes())
    
    start = nodes[0]
    unvisited = set(nodes)
    unvisited.remove(start)

    tour = [start]
    current = start

    while unvisited:
        next_node = min(unvisited, key=lambda x: G[current][x]['weight'])
        tour.append(next_node)
        unvisited.remove(next_node)
        current = next_node

    return tour

In [8]:
# EXECUCIÓ TOURS NN
tours_nn =[]
for i, G in enumerate(graphs):
    tour = nearest_neighbor_tour(G)
    print(f"Graph {i} ({G.name}):")
    print(tour)
    print()
    tours_nn.append(tour)

Graph 0 (a280):
[1, 280, 2, 3, 279, 278, 4, 277, 276, 275, 274, 273, 272, 271, 16, 17, 18, 19, 20, 21, 128, 127, 126, 125, 30, 31, 32, 29, 28, 27, 26, 22, 25, 23, 24, 14, 13, 12, 11, 10, 8, 7, 9, 6, 5, 260, 259, 258, 257, 254, 253, 208, 207, 210, 209, 252, 255, 256, 249, 248, 247, 244, 241, 240, 239, 238, 231, 232, 233, 234, 235, 236, 237, 246, 245, 243, 242, 250, 251, 230, 229, 228, 227, 226, 225, 224, 223, 222, 219, 218, 215, 214, 211, 212, 213, 216, 217, 220, 221, 203, 202, 200, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 15, 133, 132, 131, 130, 129, 154, 155, 153, 156, 152, 151, 177, 176, 181, 180, 179, 178, 150, 149, 148, 147, 146, 145, 199, 198, 197, 194, 195, 196, 201, 193, 192, 191, 190, 189, 188, 187, 185, 184, 183, 182, 161, 162, 163, 164, 165, 166, 167, 168, 169, 101, 100, 99, 98, 93, 94, 95, 96, 97, 92, 91, 90, 89, 81, 80, 79, 76, 75, 74, 73, 72, 71, 70, 67, 66, 65, 64, 63, 62, 118, 61, 60, 43, 42, 41, 40, 39, 38,

In [ ]:
# VALIDAR I GUARDAR TOURS NN
valid_tours_nn = validate_tours(tours_nn)
save_tours(valid_tours_nn, "tours_nn.txt")

Graph 0 (a280): ✅ Path vàlid
Graph 1 (ali535): ✅ Path vàlid
Graph 2 (att48): ✅ Path vàlid
Graph 3 (att532): ✅ Path vàlid
Graph 4 (bayg29): ✅ Path vàlid


# 2-opt

In [ ]:
def tour_length(G, tour):
    """Calcula la distància total del tour."""
    length = 0
    for i in range(len(tour)-1):
        length += G[tour[i]][tour[i+1]]['weight']
    return length

def two_opt(G, initial_tour):
    """
    Millora un tour utilitzant l'algorisme 2-opt.
    Retorna un nou tour.
    """
    best = initial_tour.copy()
    improved = True
    
    while improved:
        improved = False
        n = len(best)
        for i in range(1, n-1):
            for j in range(i+1, n):
                if j - i == 1:
                    continue
                new_tour = best[:i] + best[i:j][::-1] + best[j:]
                if tour_length(G, new_tour) < tour_length(G, best):
                    best = new_tour
                    improved = True
    
    return best

In [ ]:
# EXECUCIÓ TOURS 2-OPT
tours_2opt = []
for i, G in enumerate(graphs):
    nn_tour = nearest_neighbor_tour(G)
    tour_2opt = two_opt(G, nn_tour)
    print(f"Graph {i} ({G.name}):")
    print("2-opt tour:", tour_2opt)
    print()
    tours_2opt.append(tour_2opt)

Graph 0 (a280):
2-opt tour: [1, 2, 280, 3, 279, 278, 4, 5, 6, 7, 9, 8, 10, 11, 12, 13, 14, 15, 271, 16, 17, 18, 133, 132, 131, 130, 129, 154, 155, 153, 156, 152, 151, 177, 176, 181, 180, 179, 178, 150, 149, 148, 147, 146, 145, 199, 198, 197, 193, 186, 185, 184, 183, 182, 175, 160, 159, 158, 157, 119, 120, 121, 122, 123, 124, 125, 30, 126, 127, 128, 21, 20, 19, 24, 23, 25, 22, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 60, 61, 118, 117, 115, 116, 86, 85, 66, 65, 64, 63, 62, 59, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 68, 69, 67, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110, 112, 88, 83, 82, 76, 84, 87, 113, 114, 111, 107, 173, 174, 161, 162, 163, 164, 172, 171, 170, 169, 168, 167, 166, 165, 187, 188, 189, 190, 191, 192, 194, 195, 196, 201, 200, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 272,

In [ ]:
# VALIDAR I GUARDAR TOURS 2-OPT
valid_tours_2opt = validate_tours(tours_2opt)
save_tours(valid_tours_2opt, "tours_2opt.txt")

Graph 0 (a280): ✅ Path vàlid
Graph 1 (ali535): ✅ Path vàlid
Graph 2 (att48): ✅ Path vàlid
Graph 3 (att532): ✅ Path vàlid
Graph 4 (bayg29): ✅ Path vàlid


# Greedy

In [ ]:
# EXECUCIÓ TOURS GREEDY
tours_greedy = []
for i, G in enumerate(graphs):
    tour = nx.algorithms.approximation.greedy_tsp(G)
    tour = tour[:-1]
    print(f"Graph {i} ({G.name}):")
    print(tour)
    print()
    tours_greedy.append(tour)

Graph 0 (a280):
[1, 280, 2, 3, 279, 278, 4, 277, 276, 275, 274, 273, 272, 271, 16, 17, 18, 19, 20, 21, 128, 127, 126, 125, 30, 31, 32, 29, 28, 27, 26, 22, 25, 23, 24, 14, 13, 12, 11, 10, 8, 7, 9, 6, 5, 260, 259, 258, 257, 254, 253, 208, 207, 210, 209, 252, 255, 256, 249, 248, 247, 244, 241, 240, 239, 238, 231, 232, 233, 234, 235, 236, 237, 246, 245, 243, 242, 250, 251, 230, 229, 228, 227, 226, 225, 224, 223, 222, 219, 218, 215, 214, 211, 212, 213, 216, 217, 220, 221, 203, 202, 200, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 15, 133, 132, 131, 130, 129, 154, 155, 153, 156, 152, 151, 177, 176, 181, 180, 179, 178, 150, 149, 148, 147, 146, 145, 199, 198, 197, 194, 195, 196, 201, 193, 192, 191, 190, 189, 188, 187, 185, 184, 183, 182, 161, 162, 163, 164, 165, 166, 167, 168, 169, 101, 100, 99, 98, 93, 94, 95, 96, 97, 92, 91, 90, 89, 81, 80, 79, 76, 75, 74, 73, 72, 71, 70, 67, 66, 65, 64, 63, 62, 118, 61, 60, 43, 42, 41, 40, 39, 38,

In [10]:
# VALIDAR I GUARDAR TOURS GREEDY
valid_tours_greedy = validate_tours(tours_greedy)
save_tours(valid_tours_greedy, "tours_greedy.txt")

Graph 0 (a280): ✅ Path vàlid
Graph 1 (ali535): ✅ Path vàlid
Graph 2 (att48): ✅ Path vàlid
Graph 3 (att532): ✅ Path vàlid
Graph 4 (bayg29): ✅ Path vàlid
Tours saved at Datasets/tours_greedy.txt


# Christfoides

In [7]:
# EXECUCIÓ TOURS CHRISTOFIDES
tours_christofides = []
for i, G in enumerate(graphs):
    start_node = list(G.nodes())[0]
    tour = nx.algorithms.approximation.christofides(G)
    tour = tour[:-1]
    print(f"Graph {i} ({G.name}):")
    print(tour)
    print()
    tours_christofides.append(tour)

Graph 0 (a280):
[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 24, 23, 25, 22, 26, 27, 28, 29, 31, 33, 34, 36, 35, 38, 37, 39, 40, 41, 42, 43, 60, 61, 118, 62, 63, 58, 57, 44, 45, 56, 55, 46, 47, 54, 53, 48, 49, 52, 51, 50, 59, 64, 65, 85, 86, 116, 117, 115, 114, 111, 110, 108, 104, 105, 106, 107, 173, 174, 172, 171, 170, 169, 101, 102, 103, 109, 89, 81, 82, 83, 88, 112, 113, 87, 84, 66, 67, 69, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 90, 91, 92, 93, 94, 97, 96, 95, 98, 99, 100, 168, 167, 166, 165, 164, 163, 162, 161, 175, 160, 159, 158, 157, 119, 120, 121, 122, 123, 124, 125, 32, 30, 126, 127, 128, 130, 21, 20, 131, 132, 19, 129, 154, 155, 153, 156, 152, 151, 178, 177, 176, 181, 179, 180, 182, 183, 184, 185, 187, 186, 189, 188, 190, 191, 192, 193, 194, 201, 196, 195, 197, 198, 199, 145, 144, 200, 202, 203, 204, 205, 206, 207, 212, 213, 216, 217, 220, 221, 208, 209, 252, 255, 256, 248, 249, 250, 247, 244, 241, 243, 242, 240, 245, 239, 238, 231, 232, 237, 236, 233, 235, 234, 246,

In [8]:
# VALIDAR I GUARDAR TOURS CHRISTOFIDES
valid_tours_christofides = validate_tours(tours_christofides)
save_tours(valid_tours_christofides, "tours_christofides.txt")

Graph 0 (a280): ✅ Path vàlid
Graph 1 (ali535): ✅ Path vàlid
Graph 2 (att48): ✅ Path vàlid
Graph 3 (att532): ✅ Path vàlid
Graph 4 (bayg29): ✅ Path vàlid
Tours saved at Datasets/tours_christofides.txt


# Simulated Annealing

In [12]:
# EXECUCIÓ TOURS SIMULATED ANNEALING
tours_sa = []
for i, G in enumerate(graphs):
    start_node = list(G.nodes())[0]
    tour = nx.algorithms.approximation.simulated_annealing_tsp(G, nx.algorithms.approximation.greedy_tsp(G))
    tour = tour[:-1]
    print(f"Graph {i} ({G.name}):")
    print(tour)
    print()
    tours_sa.append(tour)

Graph 0 (a280):
[1, 280, 2, 3, 279, 278, 4, 277, 276, 275, 274, 273, 272, 271, 16, 17, 18, 19, 20, 21, 128, 127, 126, 125, 30, 31, 32, 29, 28, 27, 26, 22, 25, 23, 24, 14, 13, 12, 11, 10, 8, 7, 9, 6, 5, 260, 259, 258, 257, 254, 253, 208, 207, 210, 209, 252, 255, 256, 249, 248, 247, 244, 241, 240, 239, 238, 231, 232, 233, 234, 235, 236, 237, 246, 245, 243, 242, 250, 251, 230, 229, 228, 227, 226, 225, 224, 223, 222, 219, 218, 215, 214, 211, 212, 213, 216, 217, 220, 221, 203, 202, 200, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 15, 133, 132, 131, 130, 129, 154, 155, 153, 156, 152, 151, 177, 176, 181, 180, 179, 178, 150, 149, 148, 147, 146, 145, 199, 198, 197, 194, 195, 196, 201, 193, 192, 191, 190, 189, 188, 187, 185, 184, 183, 182, 161, 162, 163, 164, 165, 166, 167, 168, 169, 101, 100, 99, 98, 93, 94, 95, 96, 97, 92, 91, 90, 89, 81, 80, 79, 76, 75, 74, 73, 72, 71, 70, 67, 66, 65, 64, 63, 62, 118, 61, 60, 43, 42, 41, 40, 39, 38,

In [13]:
# VALIDAR I GUARDAR TOURS SIMULATED ANNEALING
valid_tours_sa = validate_tours(tours_sa)
save_tours(valid_tours_sa, "tours_sa.txt")

Graph 0 (a280): ✅ Path vàlid
Graph 1 (ali535): ✅ Path vàlid
Graph 2 (att48): ✅ Path vàlid
Graph 3 (att532): ✅ Path vàlid
Graph 4 (bayg29): ✅ Path vàlid
Tours saved at Datasets/tours_sa.txt


# Threshold Accepting

In [14]:
# EXECUCIÓ TOURS THRESHOLD ANNEALING
tours_ta = []
for i, G in enumerate(graphs):
    start_node = list(G.nodes())[0]
    tour = nx.algorithms.approximation.threshold_accepting_tsp(G, nx.algorithms.approximation.greedy_tsp(G))
    tour = tour[:-1]
    print(f"Graph {i} ({G.name}):")
    print(tour)
    print()
    tours_ta.append(tour)

Graph 0 (a280):
[1, 280, 2, 3, 279, 278, 4, 277, 276, 275, 274, 273, 272, 271, 16, 17, 18, 19, 20, 21, 128, 127, 126, 125, 30, 31, 32, 29, 28, 27, 26, 22, 25, 23, 24, 14, 13, 12, 11, 10, 8, 7, 9, 6, 5, 260, 259, 258, 257, 254, 253, 208, 207, 210, 209, 252, 255, 256, 249, 248, 247, 244, 241, 240, 239, 238, 231, 232, 233, 234, 235, 236, 237, 246, 245, 243, 242, 250, 251, 230, 229, 228, 227, 226, 225, 224, 223, 222, 219, 218, 215, 214, 211, 212, 213, 216, 217, 220, 221, 203, 202, 200, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 15, 133, 132, 131, 130, 129, 154, 155, 153, 156, 152, 151, 177, 176, 181, 180, 179, 178, 150, 149, 148, 147, 146, 145, 199, 198, 197, 194, 195, 196, 201, 193, 192, 191, 190, 189, 188, 187, 185, 184, 183, 182, 161, 162, 163, 164, 165, 166, 167, 168, 169, 101, 100, 99, 98, 93, 94, 95, 96, 97, 92, 91, 90, 89, 81, 80, 79, 76, 75, 74, 73, 72, 71, 70, 67, 66, 65, 64, 63, 62, 118, 61, 60, 43, 42, 41, 40, 39, 38,

In [15]:
# VALIDAR I GUARDAR TOURS THRESHOLD ANNEALING
valid_tours_ta = validate_tours(tours_ta)
save_tours(valid_tours_ta, "tours_ta.txt")

Graph 0 (a280): ✅ Path vàlid
Graph 1 (ali535): ✅ Path vàlid
Graph 2 (att48): ✅ Path vàlid
Graph 3 (att532): ✅ Path vàlid
Graph 4 (bayg29): ✅ Path vàlid
Tours saved at Datasets/tours_ta.txt
